# 案件
- 【市場調査】Amazon/ヤフオクの市場の調査をお願いします。初心者歓迎/隙間時間にもできます
- https://crowdworks.jp/public/jobs/9151993

# 動作検証

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)

In [20]:
driver.find_element(By.ID, 'twotabsearchtextbox').send_keys('Sweetwinds ペニスリング コックリング 二重リング 日本人サイズ 柔らかいシリコン 高弾力 男性用オナニーリング アダル')
driver.find_element(By.ID, 'nav-search-submit-button').click()

In [21]:
handle_array = driver.window_handles
len(handle_array)

1

In [22]:
for i in range(len(handle_array)):
    print(i, handle_array[i])

0 8C8CE6DBD3C129FD0E6B7C6D658F17B6


In [23]:
driver.find_element(By.ID, 'a-autoid-1-announce').click()

['8C8CE6DBD3C129FD0E6B7C6D658F17B6']


# 完成版
1. ヤフオクから情報を取得する
1. ヤフオクで取得した情報を基に Amazon で検索する

## ヤフオクの情報を取得する

In [ ]:
# yafuoku_crawl.py

## ヤフオクの取得データを基に Amazon の情報を検索する

In [30]:
%%time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from time import sleep
from bs4 import BeautifulSoup
import datetime
import csv
import pandas as pd

CHROMEDRIVER = r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe'
chrome_service = fs.Service(executable_path = CHROMEDRIVER)
options = Options()
# options.add_argument('--headless')  # ヘッドレスモード
driver = webdriver.Chrome(service = chrome_service, options = options)

# Amazon のトップページを開く
url = 'https://www.amazon.co.jp/ref=nav_logo'
driver.get(url)
sleep(1)

# 大元のウィンドウの保持
original_window = driver.current_window_handle

# ヤフオクデータの取得
df = pd.read_csv('yafuoku_seller_crawl.csv', encoding='cp932')

# 取得情報格納用のリスト準備
amazon_list = []

for index, yafuoku_item in df.iterrows():
    try:
        yafuoku_name = yafuoku_item['name']
        print('ヤフオク商品名:', yafuoku_name)

        yafuoku_price = yafuoku_item['price']
        # print('ヤフオク価格:', yafuoku_price)
        
        yafuoku_tax = yafuoku_item['tax']
        # print('ヤフオク税金:', yafuoku_tax)

        yafuoku_postage = yafuoku_item['postage']
        # print('ヤフオク送料:', yafuoku_postage)
        
        yafuoku_postage_due = yafuoku_item['postage_due']
        # print('ヤフオク送料負担:', yafuoku_postage_due)
        
        yafuoku_free_postage = yafuoku_item['free_postage']
        # print('ヤフオク送料無料:', yafuoku_free_postage)
        
        yafuoku_url = yafuoku_item['url']
        # print('ヤフオク商品URL:', yafuoku_url)
        
        yafuoku_end_date = yafuoku_item['end_date']
        # print('ヤフオク終了日:', yafuoku_end_date)
        
        yafuoku_info_get_date = yafuoku_item['info_get_date']
        # print('ヤフオク情報取得日:', yafuoku_info_get_date)

        # 検索ボックスに商品名を入力する
        driver.find_element(By.ID, 'twotabsearchtextbox').send_keys(yafuoku_name)
        # 検索ボタンをクリックする
        driver.find_element(By.ID, 'nav-search-submit-button').click()
        sleep(1)

        # ■ アダルト商品の場合、年齢確認のポップアップ（別ウィンドウ）が表示される
        # if driver.find_element(By.ID, 'a-autoid-1-announce'):
        #     driver.find_element(By.ID, 'a-autoid-1-announce').click()
        #     sleep(1)
        
        # 商品ページのリンクをクリックする。別ウィンドウで開かれる
        driver.find_element(By.XPATH, f'(//span[contains(text(),"{yafuoku_name}")]/parent::a[@target="_blank"])[1]').click()

        # 開いているすべてのウィンドウハンドルを取得する（リスト形式）
        handle_array = driver.window_handles
        # 別ウィンドウに切り替える
        driver.switch_to.window(handle_array[-1])
        # 別ウィンドウの HTML 構造を取得する
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        amazon_name = soup.select('span#productTitle')[0].text.strip()
        print('Amazon商品名:', amazon_name)

        amazon_price = soup.select('div#corePrice_feature_div span.a-price-whole')[0].text
        # print('Amazon価格:', amazon_price)

        amazon_url = driver.current_url
        # print('Amazon商品URL:', amazon_url)
        
        amazon_info_get_date = datetime.datetime.now()
        # print('Amazon情報取得日:', amazon_info_get_date)

        # 検索対象商品名と取得商品名が一致しているかをチェック
        if yafuoku_name == amazon_name:
            check = '一致'
        else:
            check = '【商品名不一致】'
        # print('商品名整合性:', check)

        # 取得した情報をリストに追加する
        amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_postage_due, yafuoku_free_postage, yafuoku_url, yafuoku_end_date, yafuoku_info_get_date, \
            amazon_name, amazon_price, amazon_url, amazon_info_get_date, check
        ])

        # 現在のウィンドウを閉じる
        driver.close()
        # ウィンドウハンドラを大元のウィンドウに切り替える
        driver.switch_to.window(original_window)

        # 検索ボックスに残っている文字列を消す
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()
    except:
        print('【Amazon情報を取得できませんでした】')
        check = '【Amazon情報未取得】'
        amazon_list.append([
            yafuoku_name, yafuoku_price, yafuoku_tax, yafuoku_postage, yafuoku_postage_due, yafuoku_free_postage, yafuoku_url, yafuoku_end_date, yafuoku_info_get_date, \
            '【不明】', '-', '-', '-', check
        ])
        driver.find_element(By.ID, 'twotabsearchtextbox').clear()
    finally:
        print('')

# 取得した全データを csv ファイルに出力する
csv_header = [
    'ヤフオク商品名', 'ヤフオク価格', 'ヤフオク税金', 'ヤフオク送料', 'ヤフオク送料負担', 'ヤフオク送料無料', 'ヤフオク商品URL', 'ヤフオク終了日', 'ヤフオク情報取得日', \
    'Amazon商品名', 'Amazon価格', 'Amazon商品URL', 'Amazon情報取得日', '商品名整合性'
]
csv_file_name = 'yafuoku_amazon_research.csv'

with open(csv_file_name, 'w', errors = 'ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_header)
    writer.writerows(amazon_list)
    
print('----- 取得完了 -----')

ヤフオク商品名: ABSプラスチック製　カー　フェイク　ダンプ　電子　ターボ　ブロー　オフ　フーター　バルブ　 アナログ　サウンド
Amazon商品名: 【ノーブランド 品】ABSプラスチック製　カー　フェイク　ダンプ　電子　ターボ　ブロー　オフ　フーター　バルブ　 アナログ　サウンド

ヤフオク商品名: Sweetwinds ペニスリング コックリング 二重リング 日本人サイズ 柔らかいシリコン 高弾力 男性用オナニーリング アダル
【Amazon情報を取得できませんでした】

ヤフオク商品名: Sweetwinds ペニスリング コックリング 三角リング 2点セット【黒と白】 シリコン製 男性用 アダルトグッズ
【Amazon情報を取得できませんでした】

----- 取得完了 -----
Wall time: 18.3 s
